In [2]:
# ============================================================
# Polymarket RL Trader — v9: Hybrid (API + Simulation Fallback)
# ============================================================
# Решает проблему блокировки IP на Kaggle. 
# Если API недоступно, генерирует реалистичную историю для обучения.

import os, sys, subprocess, warnings, time, json, random, math
from datetime import datetime, timezone, timedelta

# -------------------------
# 0) Installs
# -------------------------
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")

def _pip(cmd):
    subprocess.check_call([sys.executable, "-m", "pip"] + cmd)

try:
    import gymnasium
except ImportError:
    _pip(["install", "-q", "gymnasium==0.29.0", "shimmy>=0.2.1"])

try:
    import stable_baselines3
except ImportError:
    _pip(["install", "-q", "stable-baselines3==2.1.0"])

import numpy as np
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# -------------------------
# 1) Config
# -------------------------
# Ищем рынки, закрытые за последние 30 дней
LOOKBACK_DAYS = 30
MIN_DATE = (datetime.now() - timedelta(days=LOOKBACK_DAYS)).date()

MIN_VOLUME = 1000.0 
MAX_SCAN = 300 

TRAIN_STEPS = 60_000
CAPITAL = 1000.0

# API Endpoints
GAMMA_URL = "https://gamma-api.polymarket.com/markets"
CLOB_URL = "https://clob.polymarket.com/prices-history"

# Logging
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(message)s", datefmt="%H:%M:%S")
log = logging.getLogger("PolyHybrid")

# -------------------------
# 2) Data Miner
# -------------------------
class HybridMiner:
    def __init__(self):
        self.sess = requests.Session()
        retries = Retry(total=2, backoff_factor=1, status_forcelist=[500, 502, 503])
        self.sess.mount("https://", HTTPAdapter(max_retries=retries))
        # Маскируемся под браузер
        self.sess.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept": "application/json"
        })

    def fetch_markets(self):
        log.info(f"1. Сканирование рынков (Vol > {MIN_VOLUME})...")
        markets = []
        offset = 0
        
        while offset < MAX_SCAN:
            params = {"limit": 100, "offset": offset, "closed": True, "order": "volume", "ascending": False}
            try:
                r = self.sess.get(GAMMA_URL, params=params, timeout=5)
                if r.status_code != 200:
                    time.sleep(1); continue
                
                data = r.json()
                if not data: break
                
                for m in data:
                    end_str = m.get("endDate")
                    if not end_str: continue
                    end_date = pd.to_datetime(end_str).date()
                    if end_date < MIN_DATE: continue
                    
                    vol = float(m.get("volume", 0) or 0)
                    if vol < MIN_VOLUME: continue

                    try:
                        clob_ids = json.loads(m["clobTokenIds"]) if isinstance(m["clobTokenIds"], str) else m["clobTokenIds"]
                        if len(clob_ids) >= 2:
                            markets.append({
                                "id": m["id"],
                                "end_date": end_date,
                                "token_yes": clob_ids[0],
                                "volume": vol,
                                "last_price": 0.5 # Заглушка, если истории не будет
                            })
                    except:
                        continue
                offset += 100
            except Exception:
                break
        
        log.info(f"   Найдено {len(markets)} рынков.")
        return markets

    def _generate_synthetic_history(self, mkt):
        """
        Генерирует реалистичную историю, если API заблокировано.
        Использует Geometric Brownian Motion + Time Decay.
        """
        days = np.random.randint(3, 15) # Длительность рынка
        hours = days * 24
        
        # Случайный тренд: либо вверх (Yes wins), либо вниз (No wins)
        outcome = np.random.choice([0.0, 1.0], p=[0.5, 0.5])
        start_price = np.random.uniform(0.3, 0.7)
        
        # Генерируем путь
        t = np.linspace(0, 1, hours)
        # Волатильность
        sigma = 0.5 
        # Винеровский процесс
        W = np.random.standard_normal(size=hours)
        W = np.cumsum(W) * np.sqrt(1/hours)
        
        # Дрейф к исходу (outcome) по мере приближения к концу
        drift = (outcome - start_price) * t
        
        prices = start_price + drift + sigma * 0.1 * W
        prices = np.clip(prices, 0.01, 0.99)
        
        # Объемы (случайные всплески)
        volumes = np.random.lognormal(mean=np.log(1000), sigma=1.0, size=hours)
        
        # Создаем DataFrame
        dates = [pd.Timestamp(mkt['end_date']) - timedelta(hours=x) for x in range(hours)][::-1]
        
        df = pd.DataFrame({
            'p': prices,
            'v': volumes,
            'hours_left': np.arange(hours, 0, -1)
        }, index=dates)
        
        df['market_id'] = mkt['id']
        return df

    def get_data(self, markets):
        log.info("2. Получение истории цен...")
        all_dfs = []
        api_blocked = False
        
        for i, mkt in enumerate(markets):
            # Если API уже показало блок, сразу идем в симуляцию
            if not api_blocked:
                params = {"market": mkt['token_yes'], "interval": "max", "fidelity": 60}
                try:
                    time.sleep(0.3) # Rate limit
                    r = self.sess.get(CLOB_URL, params=params, timeout=5)
                    
                    if r.status_code == 200:
                        hist = r.json().get("history", [])
                        if hist:
                            df = pd.DataFrame(hist)
                            df['t'] = pd.to_datetime(df['t'], unit='s')
                            df = df.set_index('t').sort_index()
                            df['hours_left'] = (pd.to_datetime(mkt['end_date']) - df.index).total_seconds() / 3600.0
                            df = df[df['hours_left'] > 0]
                            df['market_id'] = mkt['id']
                            all_dfs.append(df[['p', 'v', 'hours_left']])
                            continue
                    elif r.status_code in [403, 429]:
                        log.warning(f"   API Blocked ({r.status_code}). Переход на симуляцию.")
                        api_blocked = True
                except Exception:
                    pass
            
            # Fallback: Симуляция
            sim_df = self._generate_synthetic_history(mkt)
            all_dfs.append(sim_df)
            
            if i % 50 == 0:
                print(f"\r   Обработано: {i}/{len(markets)}", end="")

        print("")
        if not all_dfs: return pd.DataFrame()
        return pd.concat(all_dfs)

# -------------------------
# 3) Environment
# -------------------------
class PolyEnv(gym.Env):
    def __init__(self, df):
        super().__init__()
        self.df = df
        self.episodes = [g for _, g in df.groupby('market_id')]
        self.idx = 0
        self.balance = CAPITAL
        self.shares = 0.0
        self.entry_price = 0.0
        self.action_space = spaces.Discrete(3) # Hold, Buy YES, Buy NO
        # Obs: [Bal, Pos, Price, InvPrice, Time, Vol]
        self.observation_space = spaces.Box(low=-5, high=5, shape=(6,), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.data = random.choice(self.episodes).reset_index(drop=True)
        self.idx = 0
        self.balance = CAPITAL
        self.shares = 0.0
        self.entry_price = 0.0
        return self._obs(), {}

    def _obs(self):
        row = self.data.iloc[self.idx]
        p = float(row['p'])
        return np.array([
            np.log1p(self.balance)/10.0,
            1.0 if self.shares!=0 else 0.0,
            p, 1.0-p,
            row['hours_left']/100.0,
            np.log1p(row['v'])/10.0
        ], dtype=np.float32)

    def step(self, action):
        row = self.data.iloc[self.idx]
        p = float(row['p'])
        reward = 0.0
        
        # Close
        if self.shares != 0:
            if (self.shares > 0 and action != 1) or (self.shares < 0 and action != 2):
                exit_p = p if self.shares > 0 else (1.0 - p)
                pnl = (exit_p - self.entry_price) / self.entry_price
                self.balance = abs(self.shares) * exit_p
                self.shares = 0
                reward += pnl * 10.0

        # Open
        if self.shares == 0 and self.balance > 1:
            if action == 1: # Buy YES
                self.shares = self.balance / p
                self.balance = 0
                self.entry_price = p
            elif action == 2: # Buy NO
                self.shares = -1 * (self.balance / (1.0 - p))
                self.balance = 0
                self.entry_price = 1.0 - p

        self.idx += 1
        terminated = False
        if self.idx >= len(self.data)-1:
            terminated = True
            if self.shares != 0:
                final_p = p if self.shares > 0 else (1.0 - p)
                self.balance = abs(self.shares) * final_p
                if self.balance > CAPITAL: reward += 5.0
                else: reward -= 5.0
        
        return self._obs(), reward, terminated, False, {}

# -------------------------
# 4) Main
# -------------------------
if __name__ == "__main__":
    miner = HybridMiner()
    
    # 1. Markets
    markets = miner.fetch_markets()
    
    if markets:
        # 2. Data (Hybrid)
        df = miner.get_data(markets)
        
        if not df.empty:
            log.info(f"Датасет готов: {len(df)} строк. (Real + Synthetic)")
            
            # 3. Train
            env = DummyVecEnv([lambda: PolyEnv(df)])
            model = PPO("MlpPolicy", env, verbose=1, learning_rate=3e-4)
            
            log.info("Запуск обучения...")
            model.learn(total_timesteps=TRAIN_STEPS)
            model.save("poly_agent_final")
            log.info("Модель обучена и сохранена.")
            
            # 4. Test
            obs = env.reset()
            total_rewards = 0
            for _ in range(1000):
                action, _ = model.predict(obs)
                obs, rewards, dones, _ = env.step(action)
                total_rewards += rewards
            log.info("Тестовый прогон завершен.")
            
        else:
            log.error("Не удалось сформировать данные.")
    else:
        log.error("Рынки не найдены.")

22:21:48 | 1. Сканирование рынков (Vol > 1000.0)...
22:21:48 |    Найдено 27 рынков.
22:21:48 | 2. Получение истории цен...


   Обработано: 0/27

22:21:58 | Датасет готов: 5496 строк. (Real + Synthetic)



Using cuda device


22:22:02 | Запуск обучения...


-----------------------------
| time/              |      |
|    fps             | 533  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 470         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007869378 |
|    clip_fraction        | 0.0559      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.088784754 |
|    learning_rate        | 0.0003      |
|    loss                 | 0.126       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00626    |
|    value_loss           | 0.96        |
-----------------------------------------
----------------------------------

22:24:09 | Модель обучена и сохранена.
22:24:10 | Тестовый прогон завершен.


In [3]:
# ============================================================
# Polymarket AI Sniper — Live Inference Mode
# ============================================================
# Загружает обученную модель и ищет сигналы на ЖИВОМ рынке прямо сейчас.

import pandas as pd
import numpy as np
import requests
import time
import json
from datetime import datetime, timezone
from stable_baselines3 import PPO

# 1. Загружаем модель
# Убедись, что файл poly_agent_final.zip существует в папке
try:
    model = PPO.load("poly_agent_final")
    print("✅ Модель успешно загружена.")
except:
    print("❌ Файл модели не найден. Сначала выполни обучение (предыдущий шаг).")
    exit()

# 2. Настройки сканера
MIN_VOL = 5000.0  # Ищем только ликвидность
GAMMA_URL = "https://gamma-api.polymarket.com/markets"

def get_live_markets():
    print("📡 Сканирование живого рынка...")
    sess = requests.Session()
    markets = []
    
    # Берем топ-100 активных рынков
    params = {
        "limit": 100, 
        "active": True, 
        "closed": False, 
        "order": "volume", 
        "ascending": False
    }
    
    try:
        r = sess.get(GAMMA_URL, params=params, timeout=10)
        data = r.json()
        
        for m in data:
            # Фильтры
            vol = float(m.get("volume", 0) or 0)
            if vol < MIN_VOL: continue
            
            # Парсим цену
            outcomes = json.loads(m["outcomes"]) if isinstance(m["outcomes"], str) else m["outcomes"]
            prices = json.loads(m["outcomePrices"]) if isinstance(m["outcomePrices"], str) else m["outcomePrices"]
            
            # Ищем бинарные (Yes/No)
            if len(outcomes) != 2: continue
            
            # Берем цену YES (обычно индекс 0)
            try:
                p_yes = float(prices[0])
                p_no = float(prices[1])
            except:
                continue
                
            # Время до конца
            end_date = pd.to_datetime(m.get("endDate"))
            now = pd.to_datetime(datetime.now(timezone.utc))
            if end_date.tzinfo is None: end_date = end_date.tz_localize("UTC")
            
            hours_left = (end_date - now).total_seconds() / 3600.0
            if hours_left <= 0: continue
            
            markets.append({
                "question": m["question"],
                "p_yes": p_yes,
                "p_no": p_no,
                "vol": vol,
                "hours_left": hours_left,
                "id": m["id"]
            })
            
    except Exception as e:
        print(f"Ошибка API: {e}")
        
    return pd.DataFrame(markets)

# 3. Анализ
df = get_live_markets()

if not df.empty:
    print(f"🔎 Найдено {len(df)} активных рынков. Анализируем...")
    
    signals = []
    
    for i, row in df.iterrows():
        # Формируем вектор наблюдения (как при обучении)
        # [NormBal, HasPos, Price, InvPrice, TimeNorm, LogVol]
        # Баланс ставим условный (0), позицию (0), так как мы просто смотрим сигнал
        
        obs = np.array([
            0.0, # Log Balance (start)
            0.0, # No position
            row['p_yes'],
            row['p_no'],
            row['hours_left'] / 100.0, # Norm Time
            np.log1p(row['vol']) / 10.0 # Norm Vol
        ], dtype=np.float32)
        
        # Спрашиваем модель
        action, _ = model.predict(obs, deterministic=True)
        
        # Интерпретация (0=Hold, 1=Buy YES, 2=Buy NO)
        if action == 1:
            decision = "BUY YES"
            confidence = row['p_yes'] # Условно
        elif action == 2:
            decision = "BUY NO"
            confidence = row['p_no']
        else:
            decision = "SKIP"
            confidence = 0
            
        if decision != "SKIP":
            signals.append({
                "Question": row['question'],
                "Signal": decision,
                "Price": confidence,
                "Hours Left": round(row['hours_left'], 1),
                "Volume": int(row['vol'])
            })
    
    # Вывод результатов
    if signals:
        res_df = pd.DataFrame(signals)
        # Показываем топ сигналов (где цена ниже 0.95, чтобы отсеять мусор)
        res_df = res_df[res_df["Price"] < 0.98].sort_values("Hours Left")
        
        print("\n=== 🤖 AI TRADING SIGNALS (LIVE) ===")
        pd.set_option('display.max_colwidth', 60)
        print(res_df.head(15).to_string(index=False))
    else:
        print("Модель решила, что сейчас лучше сидеть в кэше (No Signals).")

else:
    print("Не удалось получить данные с рынка.")

✅ Модель успешно загружена.
📡 Сканирование живого рынка...
Ошибка API: 'NoneType' object has no attribute 'tzinfo'
🔎 Найдено 2 активных рынков. Анализируем...

=== 🤖 AI TRADING SIGNALS (LIVE) ===
                                           Question  Signal  Price  Hours Left  Volume
Will Gold (GC) hit (LOW) $4,450 by end of February?  BUY NO  0.840       433.6   99812
                    Clavicular sentenced to prison? BUY YES  0.095      3361.6    9968
